<a href="https://colab.research.google.com/github/Merrex/Repo_Runner-CLI/blob/master/repo_runner_colab_minimal_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimal Agentic Repo Runner Test
Test any codebase with a single click.

In [1]:
# 1. Install repo_runner (latest from GitHub)
# This command always pulls the latest version from the main branch
!pip install --quiet --upgrade --force-reinstall git+https://github.com/Merrex/Repo_Runner-CLI.git

# Verify installation
import subprocess
try:
    result = subprocess.run(['repo_runner', '--version'], capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✅ repo_runner installed successfully: {result.stdout.strip()}")
    else:
        print("⚠️ repo_runner CLI not found, but Python module should be available")
except Exception as e:
    print(f"ℹ️ Installation completed (CLI check failed: {e})")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.4 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.7.1 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.1 which is incompatible.
langchain-core 0.3.67 requires packaging<25,>=23.2, but you have packaging 25.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.5.1 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have 

In [2]:
# 2. Upload and extract your test repository
from google.colab import files
import zipfile, os

uploaded = files.upload()
test_repo_path = '/content/test_repo'
for filename in uploaded:
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(test_repo_path)
        break

# 3. Find the actual repo root (where key files are)
def find_repo_root(base_path):
    key_files = ['package.json', 'requirements.txt', 'docker-compose.yml', 'main.py', 'app.py', 'index.js', 'App.jsx']
    for file in key_files:
        if os.path.exists(os.path.join(base_path, file)):
            return base_path
    for root, dirs, files in os.walk(base_path):
        for file in key_files:
            if file in files:
                return root
    return base_path

actual_repo_path = find_repo_root(test_repo_path)
print(f'Repo root: {actual_repo_path}')


Saving realtime-chat-gemini-main.zip to realtime-chat-gemini-main (2).zip
Repo root: /content/test_repo/realtime-chat-gemini-main


In [3]:
# 4. Run repo_runner fully agentically
import subprocess
result = subprocess.run(['repo_runner', 'run', actual_repo_path, '--mode', 'local'], capture_output=True, text=True)
print('Exit code:', result.returncode)
print('--- STDOUT ---')
print(result.stdout[-2000:])  # Show last 2000 chars
print('--- STDERR ---')
print(result.stderr[-2000:])  # Show last 2000 chars


Exit code: 1
--- STDOUT ---

--- STDERR ---
^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torchvision/_meta_registrations.py", line 163, in <module>
    @torch.library.register_fake("torchvision::nms")
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/library.py", line 1023, in register
    use_lib._register_fake(op_name, func, _stacklevel=stacklevel + 1)
  File "/usr/local/lib/python3.11/dist-packages/torch/library.py", line 214, in _register_fake
    handle = entry.fake_impl.register(func_to_register, source)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/_library/fake_impl.py", line 31, in register
    if torch._C._dispatch_has_kernel_for_dispatch_key(self.qualname, "Meta"):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: operator torchvision::nms does not exist

The above exception was the direc